In [129]:
import pandas as pd
import numpy as np
import os
import openpyxl 

In [130]:
import pandas as pd

def excel_to_df(excel_file: str, sheet_name: str = None) -> pd.DataFrame:

    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
    )
    
    df = df.convert_dtypes()
    
    return df

In [ ]:
xml_df = excel_to_df("#FILE PATH HERE", sheet_name="Annual")

xml_df.head()

,observation_date,mehoinusa672n
0,1984-01-01,60420
1,1985-01-01,61570
2,1986-01-01,63850
3,1987-01-01,64650
4,1988-01-01,65130


In [ ]:
csv_df = pd.read_csv("#FILE PATH HERE", encoding="utf-8")
csv_df.head(3)

C:\Users\chris\AppData\Local\Temp\ipykernel_17032\4153003517.py:1: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv("C:\\Users\\chris\\Downloads\\is project\\Real_Estate_Sales_2001-2023_GL.csv", encoding="utf-8")


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463,Commercial,NaN,NaN,NaN,NaN,NaN


In [133]:
xml_df.columns

Index(['observation_date', 'mehoinusa672n'], dtype='object')

In [134]:
csv_df.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type', 'Non Use Code', 'Assessor Remarks', 'OPM remarks',
       'Location'],
      dtype='object')

In [135]:
print(xml_df.shape)
print(csv_df.shape)


(41, 2)
(1141722, 14)


In [136]:
csv_df["Date Recorded"] = pd.to_datetime(csv_df["Date Recorded"], format="%m/%d/%Y", errors="coerce")
xml_df["observation_date"] = pd.to_datetime(xml_df["observation_date"], errors="coerce")
xml_df["Year"] = xml_df["observation_date"].dt.year
csv_df = csv_df.rename(columns={"List Year": "Year"})

In [137]:
csv_df.head(2)

,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,2021-04-14,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,2021-05-26,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN


In [138]:
merged_df = csv_df.merge(
    xml_df[["Year", "mehoinusa672n"]],
    on="Year",
    how="left"
)

In [139]:
merged_df.head(4)

,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,mehoinusa672n
0,2020177,2020,2021-04-14,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014),81580
1,2020225,2020,2021-05-26,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN,81580
2,2020348,2020,2021-09-13,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463,Commercial,NaN,NaN,NaN,NaN,NaN,81580
3,2020090,2020,2020-12-14,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN,81580


In [140]:
# Since we are mostly focusing on pre an during covid, we will be focusing only on 2015 to 2023

In [141]:
merged_df["Date"] = merged_df["Date Recorded"]
merged_df = merged_df.drop(columns=["Non Use Code", "Assessor Remarks", "OPM remarks", "Location", "Date Recorded"])
merged_df = merged_df.sort_values("Year")

In [142]:
filtered_df = merged_df[(merged_df["Year"] >= 2015) & (merged_df["Year"] <= 2023)]

In [143]:
filtered_df.head()

,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
827798,150096,2015,Milford,61 SPICE BUSH LN,317600.0,235500.0,1.34862,Condo,Condo,72790,2016-08-31
828446,150854,2015,Waterbury,14 JODIE CIRCLE,69830.0,75900.0,0.920026,Single Family,Single Family,72790,2016-05-02
828068,15456,2015,Hartford,113-115 BURNHUM ST,49733.0,103950.0,0.478432,Two Family,Two Family,72790,2016-04-28
828440,150310,2015,Cheshire,1281 AVON BOULEVARD,144080.0,223000.0,0.646099,Single Family,Single Family,72790,2016-06-17
828032,150043,2015,Middlebury,250 WHITE DEER ROCK RD,331800.0,480000.0,0.69125,Single Family,Single Family,72790,2016-01-08


In [144]:
filtered_df.shape

(462879, 11)

In [145]:
missing_rows = filtered_df[filtered_df.isna().any(axis=1)]
print(missing_rows.shape)
missing_rows.head(2)

(41212, 11)


,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
828038,150027,2015,Litchfield,TALLMADGE LANE,127730.0,250000.0,0.51092,NaN,NaN,72790,2015-11-25
828583,150185,2015,Cromwell,46 PASCO HILL ROAD,117480.0,127500.0,0.921412,NaN,NaN,72790,2016-06-16


In [104]:
filtered_df.isna().any()

Serial Number       False
Year                False
Date Recorded        True
Town                False
Address              True
Assessed Value      False
Sale Amount         False
Sales Ratio         False
Property Type        True
Residential Type     True
mehoinusa672n       False
Date                 True
dtype: bool

In [124]:
print(filtered_df[filtered_df["Property Type"].isna() & filtered_df["Residential Type"].isna()].shape)
filtered_df[filtered_df["Property Type"].isna() & filtered_df["Residential Type"].isna()].head(2)

(20525, 12)


,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
828038,150027,2015,2015-11-25,Litchfield,TALLMADGE LANE,127730.0,250000.0,0.51092,NaN,NaN,72790,2015-11-25
828583,150185,2015,2016-06-16,Cromwell,46 PASCO HILL ROAD,117480.0,127500.0,0.921412,NaN,NaN,72790,2016-06-16


In [127]:
filtered_df = filtered_df.dropna(subset=["Date Recorded"])
Cleaned_Df = filtered_df.dropna(subset=["Property Type", "Residential Type"], how="all")

In [128]:
print(Cleaned_Df.shape)
Cleaned_Df.head(2)

(442105, 12)


,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
827798,150096,2015,2016-08-31,Milford,61 SPICE BUSH LN,317600.0,235500.0,1.34862,Condo,Condo,72790,2016-08-31
828446,150854,2015,2016-05-02,Waterbury,14 JODIE CIRCLE,69830.0,75900.0,0.920026,Single Family,Single Family,72790,2016-05-02
